# Feature Engineering

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from ml2_mlsupervisadoavanzado.eee.feature_enginering import FeatureEngineeringProcessor

2025-08-03 14:43:35.222 | INFO     | ml2_mlsupervisadoavanzado.config:<module>:11 - PROJ_ROOT path is: /home/edyn/Documentos/obsidian-doc/Projects/MachineLearningEngineer/Curso2/Proyecto/Repo/ml2_class4_mlsupervisadoavanzado


In [2]:
# Dataset Hotel Booking -> https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand/data
raw_data = pd.read_csv("../data/raw/hotel_bookings.csv")

In [3]:
# Divide el DataFrame en conjuntos de entrenamiento y prueba usando scikit-learn.
train_raw_data, test_raw_data = train_test_split(
    raw_data, test_size=0.2, random_state=42
)

In [4]:
train_processor = FeatureEngineeringProcessor(
    raw_data=train_raw_data,
    pipeline_name="train_pipeline"
)
# Ejecuta el procesamiento de características.
train_processor.run()

# Guarda la tabla de características en disco.
train_processor.write_feature_table("../ml2_mlsupervisadoavanzado/eee/data/bookings_feature_table")

2025-08-03 14:43:40.064 | INFO     | ml2_mlsupervisadoavanzado.eee.feature_enginering:run:68 - Inicializando pipeline train_pipeline
2025-08-03 14:43:40.065 | INFO     | ml2_mlsupervisadoavanzado.eee.feature_enginering:encode_categoricals:51 - Codificando con OHE hotel
2025-08-03 14:43:40.083 | INFO     | ml2_mlsupervisadoavanzado.eee.feature_enginering:encode_categoricals:51 - Codificando con OHE market_segment
2025-08-03 14:43:40.103 | INFO     | ml2_mlsupervisadoavanzado.eee.feature_enginering:encode_categoricals:51 - Codificando con OHE reserved_room_type
2025-08-03 14:43:41.773 | INFO     | ml2_mlsupervisadoavanzado.eee.feature_enginering:write_feature_table:108 - Escribiendo feature table en ../ml2_mlsupervisadoavanzado/eee/data/bookings_feature_table


In [5]:
df_parquet = pd.read_parquet("../ml2_mlsupervisadoavanzado/eee/data/bookings_feature_table.parquet")
df_parquet[["booking_id", "event_timestamp", "created", "great_feature1", "great_feature2"]].head()

,booking_id,event_timestamp,created,great_feature1,great_feature2
0,cdf7bb09-fe23-4702-b7e8-58dd3c575223,2025-08-03 14:43:40.582836,2025-08-03 14:43:41.681146,2.169662,0.890985
1,2fb165e2-c3f3-428a-a378-48d5edb30cf4,2025-08-03 14:43:40.582840,2025-08-03 14:43:41.681151,2.309743,-0.817679
2,0f766ae4-d10e-4be4-9c61-54706f33c77c,2025-08-03 14:43:40.582840,2025-08-03 14:43:41.681151,5.391121,-1.749610
3,b5affb77-14b4-46f4-abf8-5979a50bec0b,2025-08-03 14:43:40.582840,2025-08-03 14:43:41.681152,-0.049387,1.275659
4,d543d17e-b29e-4659-9772-514806bd2dc6,2025-08-03 14:43:40.582841,2025-08-03 14:43:41.681152,-0.626450,-0.132513


In [20]:
df_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95512 entries, 0 to 95511
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   hotel_City Hotel              95512 non-null  float64       
 1   hotel_Resort Hotel            95512 non-null  float64       
 2   market_segment_Aviation       95512 non-null  float64       
 3   market_segment_Complementary  95512 non-null  float64       
 4   market_segment_Corporate      95512 non-null  float64       
 5   market_segment_Direct         95512 non-null  float64       
 6   market_segment_Groups         95512 non-null  float64       
 7   market_segment_Offline TA/TO  95512 non-null  float64       
 8   market_segment_Online TA      95512 non-null  float64       
 9   market_segment_Undefined      95512 non-null  float64       
 10  reserved_room_type_A          95512 non-null  float64       
 11  reserved_room_type_B        

# Consumir features

In [38]:
from feast import FeatureStore
from feast import FileSource, Entity, FeatureView, ValueType, Field
from feast.types import Float64

# Define una entidad para los clientes.
customer_entity = Entity(
    name="customer_id",
    value_type=ValueType.STRING,
    join_keys=["booking_id"],
    description="ID del booking del cliente",
)
## Define una nueva fuente de datos
#customer_source = FileSource(
    #name="customer_source",
    #path="../../ml2_mlsupervisadoavanzado/eee/data/bookings_feature_table.parquet",
    #timestamp_field="event_timestamp",
#)
# Define una vista de características para los clientes.
customer_view = FeatureView(
    name="customer_view",
    entities=[customer_entity],
    source=FileSource(
        name="customer_source",
        path="../../ml2_mlsupervisadoavanzado/eee/data/bookings_feature_table.parquet",
        timestamp_field="event_timestamp",
    ),
    schema=[
        Field(name="great_feature1", dtype=Float64, description="This is a great feature"),
        Field(name="great_feature2", dtype=Float64, description="This is a great feature"),
    ],
    # ttl=None,  # No se aplica TTL para este ejemplo.
    online=True,
)

# Crea un FeatureStore para consumir las características.
fs = FeatureStore("../feature_repo/feature_repo")
# Registra la entidad y la vista de características en el FeatureStore.
fs.apply([customer_entity, customer_view])

In [ ]:
## Carga las características de la vista de clientes.
#features = fs.get_online_features(
    #features=["customer_view:great_feature1", "customer_view:great_feature2"],
    ## entity_rows=[{"customer_id": 1}, {"customer_id": 2}],
    #entity_rows= [{"customer_id": i} for i in pd.read_parquet("../ml2_mlsupervisadoavanzado/eee/data/bookings_feature_table.parquet")["booking_id"][:1].tolist()]
#).to_dict()
## Imprime las características obtenidas.
#print(features)

{'booking_id': ['cdf7bb09-fe23-4702-b7e8-58dd3c575223'], 'great_feature2': [None], 'great_feature1': [None]}


In [42]:
entity_df_offline = pd.DataFrame.from_dict(
        {
            # IDs de reservas para consultar features.
            # "customer_id": ["f1ccaa85-7747-4dc3-be4d-5f229a2e07a4", "1466b87e-16c3-484e-b122-c0de8f375e0e"],
            "booking_id": pd.read_parquet("../ml2_mlsupervisadoavanzado/eee/data/bookings_feature_table.parquet")["booking_id"][:1000].tolist(),
        },
    )

entity_df_offline["event_timestamp"] = pd.to_datetime("now", utc=True)
entity_df_offline

,booking_id,event_timestamp
0,cdf7bb09-fe23-4702-b7e8-58dd3c575223,2025-08-03 20:20:38.380553+00:00
1,2fb165e2-c3f3-428a-a378-48d5edb30cf4,2025-08-03 20:20:38.380553+00:00
2,0f766ae4-d10e-4be4-9c61-54706f33c77c,2025-08-03 20:20:38.380553+00:00
3,b5affb77-14b4-46f4-abf8-5979a50bec0b,2025-08-03 20:20:38.380553+00:00
4,d543d17e-b29e-4659-9772-514806bd2dc6,2025-08-03 20:20:38.380553+00:00
...,...,...
995,bc8b9764-44f4-495d-b2e6-494eaceeb5d3,2025-08-03 20:20:38.380553+00:00
996,24dc9098-6b84-4be5-af6e-c9f70f840c6e,2025-08-03 20:20:38.380553+00:00
997,c1570dbd-4699-4e3d-98a6-65d5b808efc6,2025-08-03 20:20:38.380553+00:00
998,4830f817-113b-4ba5-9f81-1069f768c4cf,2025-08-03 20:20:38.380553+00:00


In [43]:
fs.get_historical_features(
    # Consulta las features históricas en Feast usando los valores del DataFrame.
    entity_df=entity_df_offline,
    features=[
        # Solicita las features 'great_feature1' y 'great_feature2' de la vista 'pc_booking_view',
        "customer_view:great_feature1",
        "customer_view:great_feature2",
    ],
# Convierte el resultado a un DataFrame de pandas.
).to_df()

,booking_id,event_timestamp,great_feature1,great_feature2
0,cdf7bb09-fe23-4702-b7e8-58dd3c575223,2025-08-03 20:20:38.380553+00:00,2.169662,0.890985
1,2fb165e2-c3f3-428a-a378-48d5edb30cf4,2025-08-03 20:20:38.380553+00:00,2.309743,-0.817679
2,0f766ae4-d10e-4be4-9c61-54706f33c77c,2025-08-03 20:20:38.380553+00:00,5.391121,-1.749610
3,b5affb77-14b4-46f4-abf8-5979a50bec0b,2025-08-03 20:20:38.380553+00:00,-0.049387,1.275659
4,d543d17e-b29e-4659-9772-514806bd2dc6,2025-08-03 20:20:38.380553+00:00,-0.626450,-0.132513
...,...,...,...,...
995,24dc9098-6b84-4be5-af6e-c9f70f840c6e,2025-08-03 20:20:38.380553+00:00,-0.573657,-0.360897
996,11becced-3178-4025-b5b5-ef3feacaecdb,2025-08-03 20:20:38.380553+00:00,-0.324688,-0.825786
997,c1570dbd-4699-4e3d-98a6-65d5b808efc6,2025-08-03 20:20:38.380553+00:00,-2.080054,0.696878
998,4830f817-113b-4ba5-9f81-1069f768c4cf,2025-08-03 20:20:38.380553+00:00,0.347920,0.965046
